In [1]:
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

# stopwords for English
import nltk
from nltk.corpus import stopwords

# English 'dictionary'
from nltk.corpus import words

# Regular expressions might be useful
import re

# Beautiful soup might be useful
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

To keep the project in scope (i.e. feasible given our time period), I propose that the only score column we take into account is domain1 --- all essays have this column, and I don't think the variance amongst scorers, or specific trait scoring of essays, is practical for us in this project. 

In [2]:
# Read in training data
train_cols = ['essay_id', 'essay_set', 'essay', 'domain1_score']
train_df = pd.read_csv('data/training_set_rel3.tsv', delimiter='\t', usecols=train_cols)
print train_df.head()

   essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   domain1_score  
0              8  
1              9  
2              7  
3             10  
4              8  


In [8]:
# Show nothing is empty in training set
if train_df.isnull().any().any():
    print 'Data is missing!'
else:
    print 'No missing data!'

No missing data!


In [3]:
# Read in validation data
valid_cols = ['essay_id', 'essay_set', 'essay', 'domain1_predictionid']
valid_df = pd.read_csv('data/valid_set.tsv', delimiter='\t', usecols=valid_cols)
print valid_df.head()

   essay_id  essay_set                                              essay  \
0      1788          1  Dear @ORGANIZATION1, @CAPS1 more and more peop...   
1      1789          1  Dear @LOCATION1 Time @CAPS1 me tell you what I...   
2      1790          1  Dear Local newspaper, Have you been spending a...   
3      1791          1  Dear Readers, @CAPS1 you imagine how life woul...   
4      1792          1  Dear newspaper, I strongly believe that comput...   

   domain1_predictionid  
0                  1788  
1                  1789  
2                  1790  
3                  1791  
4                  1792  


In [9]:
# Show nothing is empty in validation set
if valid_df.isnull().any().any():
    print 'Data is missing!'
else:
    print 'No missing data!'

No missing data!


In [12]:
# maps validations set domain1_predictionid to a score
# note that scoring system is different for each essay set
valid_scores = pd.read_csv('data/valid_sample_submission_2_column.csv', delimiter=',')
print valid_scores.head()

   prediction_id  predicted_score
0           1788                7
1           1789                8
2           1790                9
3           1791                9
4           1792                9


In [13]:
# Show nothing is empty in validation scores
if valid_scores.isnull().any().any():
    print 'Data is missing!'
else:
    print 'No missing data!'

No missing data!
